# Лабораторная работа 4. Часть 1. Ранжирование.



Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 9 баллов. Сдавать задание после указанного в lk срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).



## Знакомство с данными

### Ранжирование организаций по пользовательскому запросу

Что мы обычно делаем, когда нам нужно найти определённое место, но не знаем его местоположения? Используем поиск на картах.

В этой лабораторной работе вам будет необходимо построить небольшую поисковую систему, позволяющую отранжировать организации по запросу пользователя.

Для обучения вам даны 2000 запросов и более 13 тысяч найденных по ним организаций. Для каждой пары "запрос — организация" была посчитана релевантность, по которой и происходит ранжирование.

**(2 балла) Задание 1.** Загрузите [данные](https://disk.yandex.ru/d/Bf3P4H8FDYe7-g) о запросах и их релевантности (*train.csv*), а также информацию об организациях (*train_org_information.json*) и рубриках (*train_rubric_information.json*)

Для дальнейших экспериментов необходимо посчитать небольшой набор базовых факторов. С использованием информации о запросах и организациях, посчитайте факторы, которые на ваш взгляд будут полезными для предсказания релевантности.

Примерами текстовых факторов могут служить:
 - кол-во слов в запросе и названии организации;
 - пословные/N-граммные пересечения слов запроса и названия организации (также можно использовать синонимы названия организации и адрес организации): кол-во слов в пересечении, [мера Жаккара](https://en.wikipedia.org/wiki/Jaccard_index) и пр.;
 - кол-во различных синонимичных названий организации (поле *names* в описании организации);
 - One-hot-encoded язык запроса.
 
По информации о географическом положении:
 - факт совпадения региона, где задавался запрос и региона организации;
 - координаты показанной области;
 - размеры показанной области;
 - меры, характеризующие близость координат организации к показанному окну: расстояние до центра области и другие.
 
Факторы, описывающие организацию:
 - one-hot-encoding фактор cтраны или региона организации (важно: не используйте one-hot-encoding факторы, в которых больше 10 значений; если в факторе слишком много значений, ограничьтесь, например, только самыми популярными категориями)
 - кол-во рабочих дней в неделе и общая продолжительность работы (поле *work_intervals* в описании организации)
 - кол-во рубрик (поле *rubrics* в описании организации)
 
![](https://miro.medium.com/max/1500/0*FwubnnoNlt6Coo9j.png)

В этом задании не нужно использовать многомерные представления текстовой информации (tf-idf и прочие embeddings) и информацию о кликах (*train_clicks_information.json*). Придумывать сверхсложные факторы тоже необязательно.

Вы можете реализовать описанные выше факторы и/или придумать свои. Но зачастую такие простые признаки могут приносить наибольшую пользу.

В итоге у вас должно получиться от 15 до 50 факторов, характеризующих запрос и организацию и покрывающих основные источники данных (кроме кликов). Это наш основной датасет, который будет использоваться в экспериментах.



In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
!cp drive/MyDrive/Colab\ Notebooks/ML-2/Lab\ 4/2021_sem3_lab_04_part_01_ranking_data.zip data.zip
!unzip data.zip

Archive:  data.zip
   creating: 2021_sem3_lab_04_part_01_ranking_data/
  inflating: 2021_sem3_lab_04_part_01_ranking_data/train.csv  
   creating: 2021_sem3_lab_04_part_01_ranking_data/train_clicks_information/
  inflating: 2021_sem3_lab_04_part_01_ranking_data/train_clicks_information/train_clicks_information.json  
   creating: 2021_sem3_lab_04_part_01_ranking_data/train_org_information/
  inflating: 2021_sem3_lab_04_part_01_ranking_data/train_org_information/train_org_information.json  
   creating: 2021_sem3_lab_04_part_01_ranking_data/train_rubric_information/
  inflating: 2021_sem3_lab_04_part_01_ranking_data/train_rubric_information/train_rubric_information.json  


In [3]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

In [4]:
path = '2021_sem3_lab_04_part_01_ranking_data/'
train = pd.read_csv(path+'train.csv')
train.head(2)

,query_id,query,region,org_name,org_id,window_center,window_size,relevance
0,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Суд Жовтневого району міста Дніпропетровськ,1021049127,"34.613119,48.506531","0.025928,0.017380",0.0
1,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Дніпропетровський окружний адміністративний суд,1602348889,"34.613119,48.506531","0.025928,0.017380",0.0


In [5]:
import json

with open(path+'train_rubric_information/train_rubric_information.json') as f:
    rubric = json.load(f)

with open(path+'train_org_information/train_org_information.json') as f:
    org = json.load(f)

with open(path+'train_clicks_information/train_clicks_information.json') as f:
    clicks = json.load(f)

### Org

In [6]:
len(org)

27697

In [7]:
list(org.keys())[:5]

['1255014404', '1111883782', '39713767434', '1336016908', '149331116064']

In [7]:
org['1255014404'].keys()

dict_keys(['names', 'rubrics', 'work_intervals', 'address'])

In [8]:
org['1255014404']['names']

[{'value': {'locale': 'en', 'value': 'Bely Boks'}},
 {'value': {'locale': 'ru', 'value': 'Белый бокс'}}]

In [9]:
org['1255014404']['rubrics']

[20277, 20679, 21237]

In [10]:
org['1255014404']['work_intervals']

[{'day': 'everyday', 'time_minutes_begin': 540, 'time_minutes_end': 1260}]

In [11]:
org['1255014404']['address']

{'formatted': {'locale': 'ru',
  'value': 'Россия, Санкт-Петербург, Кронштадтская улица, 19'},
 'geo_id': 2,
 'pos': {'coordinates': [30.257127, 59.865881], 'type': 'Point'},
 'region_code': 'RU'}

### Rubric


In [12]:
len(rubric)

1080

In [8]:
list(rubric.keys())[:5]

['273147906', '30723', '30724', '30725', '30731']

In [14]:
rubric['273147906'].keys()

dict_keys(['keywords', 'phrases', 'descriptions', 'names'])

In [15]:
rubric['273147906']['keywords']

[{'locale': 'ru', 'value': 'велосипед парковка стоянка велосипедный вело'}]

In [16]:
rubric['273147906']['phrases']

[{'locale': 'de', 'value': 'Parkplatz für Fahrräder'},
 {'locale': 'en', 'value': 'bicycle parking'},
 {'locale': 'en', 'value': 'bicycle stand'},
 {'locale': 'en', 'value': 'bike parking'},
 {'locale': 'en', 'value': 'bike stand'},
 {'locale': 'tr', 'value': 'bisiklet bırakılacak yerler'},
 {'locale': 'tr', 'value': 'bisiklet park yeri'},
 {'locale': 'tr', 'value': 'bisiklet park yerleri'},
 {'locale': 'fr', 'value': 'parking de vélo'},
 {'locale': 'fr', 'value': 'parking pour les vélos'},
 {'locale': 'fr', 'value': 'parking à vélos'},
 {'locale': 'ru', 'value': 'велопарковка'},
 {'locale': 'uk', 'value': 'велопарковка'},
 {'locale': 'uk', 'value': 'велосипедна парковка'},
 {'locale': 'uk', 'value': 'велосипедна стоянка'},
 {'locale': 'ru', 'value': 'велосипедная парковка'},
 {'locale': 'ru', 'value': 'велосипедная стоянка'},
 {'locale': 'ru', 'value': 'парковка для велосипедов'},
 {'locale': 'uk', 'value': 'парковка для велосипедів'},
 {'locale': 'ru', 'value': 'стоянка для велосипед

In [17]:
rubric['273147906']['descriptions']

[{'value': {'locale': 'tr', 'value': 'Bisiklet park yerleri.'}},
 {'value': {'locale': 'ru', 'value': 'Места парковки велосипедов.'}},
 {'value': {'locale': 'ru', 'value': 'Места парковки велосипедов.\n\n'}}]

In [18]:
rubric['273147906']['names']

[{'locale': 'en', 'value': 'Bicycle stand'},
 {'locale': 'tr', 'value': 'Bisiklet park yerleri'},
 {'locale': 'de', 'value': 'Fahrradständer'},
 {'locale': 'it', 'value': 'Parcheggio di biciclette'},
 {'locale': 'fr', 'value': 'Parking pour vélos'},
 {'locale': 'kk', 'value': 'Велоорынтұрақ'},
 {'locale': 'ru', 'value': 'Велопарковка'},
 {'locale': 'uk', 'value': 'Велопарковка'},
 {'locale': 'uz', 'value': 'Велосипедлар турар жойи'},
 {'locale': 'hy', 'value': 'Հեծանիվների կայան'},
 {'locale': 'fa', 'value': 'پارکینگ دوچرخه'}]

### Добавление факторов

In [19]:
dataset = train.copy()

__Текстовые факторы__

Реализовано:

* кол-во слов в запросе
* кол-во слов в названии организации
* кол-во различных синонимичных названий организации
* мера Жаккара для
  * слов запроса и названия организации
  * слов запроса и наиболее похожего синонима названия организации
  * слов запроса и адреса организации

In [20]:
import re
import string

def tokenize(text):
    tokens = re.sub('\W+', ' ', text.lower()).split()
    return tokens

def jaccard_index(tokens_1, tokens_2):
    set_1, set_2 = set(tokens_1), set(tokens_2)
    return len(set_1.intersection(set_2)) / len(set_1.union(set_2))

def get_address(org_id):
    return org[str(org_id)]['address']['formatted']['value']

def get_synonyms(org_id):
    return [name['value']['value'] for name in org[str(org_id)]['names']]

In [21]:
%%time
dataset['query_len'] = dataset['query'].apply(lambda x: len(tokenize(x)))
dataset['org_name_len'] = dataset['org_name'].apply(lambda x: len(tokenize(x)))
dataset['org_name_synonyms'] = dataset['org_id'].apply(lambda x: len(org[str(x)]['names']))
dataset['jaccard_index_name'] = dataset.apply(lambda x: jaccard_index(tokenize(x['query']), 
                                                                      tokenize(x['org_name'])), axis=1)
dataset['jaccard_index_synonyms'] = dataset.apply(lambda x: max([jaccard_index(tokenize(x['query']), tokenize(syn))
                                                                 for syn in get_synonyms(x['org_id'])]), axis=1)
dataset['jaccard_index_address'] = dataset.apply(lambda x: jaccard_index(tokenize(x['query']), 
                                                                         tokenize(get_address(x['org_id']))), axis=1)


CPU times: user 4.12 s, sys: 35.3 ms, total: 4.15 s
Wall time: 4.16 s


__Факторы из географического положения__

Реализовано:

* флаг совпадения региона запроса и региона организации
* расстояние от координат организации до центра показанной области
* площадь показанной области
* флаг покрытия точки с организацией показанной областью

In [22]:
from geopy.distance import geodesic

def geo_match(region, org_id):
    return int(region == org[str(org_id)]['address']['geo_id'])

def clip_coord(lat, long):
    return np.clip(lat, -90, 90), np.clip(long, -180, 180)

def get_distance(window_center, org_id):
    coord_1 = tuple(reversed(list(map(float, window_center.split(',')))))
    coord_2 = tuple(reversed(org[str(org_id)]['address']['pos']['coordinates']))
    return geodesic(coord_1, coord_2).km

def get_window_borders(window_center, window_size):
    window_center_lat, window_center_long = reversed(list(map(float, window_center.split(','))))
    delta_lat, delta_long = reversed(list(map(float, window_size.split(','))))
    window_borders_lat = (window_center_lat - delta_lat / 2, window_center_lat + delta_lat / 2)
    window_borders_long = (window_center_long - delta_long / 2, window_center_long + delta_long / 2)
    return window_borders_lat, window_borders_long

def get_window_square(window_center, window_size):
    window_borders_lat, window_borders_long = get_window_borders(window_center, window_size)
    lat_length = geodesic(clip_coord(window_borders_lat[1], window_borders_long[0]), 
                          clip_coord(window_borders_lat[1], window_borders_long[1])).km
    long_length = geodesic(clip_coord(window_borders_lat[1], window_borders_long[0]), 
                           clip_coord(window_borders_lat[0], window_borders_long[0])).km
    return lat_length * long_length

def point_within_window(window_center, window_size, org_id):
    point = tuple(reversed(org[str(org_id)]['address']['pos']['coordinates']))
    window_borders_lat, window_borders_long = get_window_borders(window_center, window_size)
    return int(
        (point[0] >= window_borders_lat[0]) & 
        (point[0] <= window_borders_lat[1]) & 
        (point[1] >= window_borders_long[0]) & 
        (point[1] <= window_borders_long[1])
    )

In [23]:
%%time
dataset['geo_match'] = dataset.apply(lambda x: geo_match(x['region'], x['org_id']), axis=1)
dataset['distance_km'] = dataset.apply(lambda x: get_distance(x['window_center'], x['org_id']), axis=1)
dataset['window_square'] = dataset.apply(lambda x: get_window_square(x['window_center'], x['window_size']), axis=1)
dataset['point_in_window'] = dataset.apply(lambda x: point_within_window(x['window_center'], 
                                                                         x['window_size'], 
                                                                         x['org_id']), axis=1)

CPU times: user 29.7 s, sys: 126 ms, total: 29.8 s
Wall time: 32.9 s


__Факторы из организации__

Реализовано:

* one-hot-encoding региона организации
* кол-во рабочих дней в неделе
* общая продолжительность работы
* кол-во рубрик

In [24]:
from collections import Counter
from sklearn.preprocessing import OneHotEncoder

region_count = dict(Counter([x[1]['address']['region_code'] for x in org.items()]))
region_count_df = pd.DataFrame({'region_code': region_count.keys(), 
                                'count': region_count.values()}).sort_values(by='count', ascending=False)
top10_regions = region_count_df.head(10)['region_code'].values

def get_region_code(org_id, limited_regions=top10_regions):
    region = org[str(org_id)]['address']['region_code']
    if region in limited_regions:
        return region
    else:
        return 'OTHER'

def ohe_region_code(dataset):
    encoder = OneHotEncoder()
    encoded = encoder.fit_transform(dataset[['region_code']]).toarray()
    encoded_df = pd.DataFrame(encoded, columns=[f'region_ohe_{i}' for i in range(encoded.shape[1])])
    return pd.concat([dataset, encoded_df], axis=1)

days_count_mapping = {'everyday': 7, 
                      'weekdays': 5, 
                      'weekend': 2, 
                      'sunday': 1, 
                      'monday': 1, 
                      'tuesday': 1, 
                      'wednesday': 1, 
                      'thursday': 1, 
                      'friday': 1, 
                      'saturday': 1}

def get_num_working_days(org_id):
    intervals = org[str(org_id)]['work_intervals']
    if len(intervals) == 0:
        return 0
    working_days = list(set([i['day'] for i in intervals]))
    return sum([days_count_mapping[day] for day in working_days])

def get_total_working_time(org_id):
    intervals = org[str(org_id)]['work_intervals']
    if len(intervals) == 0:
        return 0
    return sum([i['time_minutes_end'] - i['time_minutes_begin'] for i in intervals])

In [25]:
%%time
dataset['rubric_count'] = dataset['org_id'].apply(lambda x: len(org[str(x)]['rubrics']))
dataset['num_working_days'] = dataset['org_id'].apply(get_num_working_days)
dataset['total_working_time'] = dataset['org_id'].apply(get_total_working_time)
dataset['region_code'] = dataset['org_id'].apply(get_region_code)
dataset = ohe_region_code(dataset)

CPU times: user 483 ms, sys: 9 ms, total: 492 ms
Wall time: 498 ms


---

In [26]:
dataset.tail(3)

,query_id,query,region,org_name,org_id,window_center,window_size,relevance,query_len,org_name_len,org_name_synonyms,jaccard_index_name,jaccard_index_synonyms,jaccard_index_address,geo_match,distance_km,window_square,point_in_window,rubric_count,num_working_days,total_working_time,region_code,region_ohe_0,region_ohe_1,region_ohe_2,region_ohe_3,region_ohe_4,region_ohe_5,region_ohe_6,region_ohe_7,region_ohe_8,region_ohe_9,region_ohe_10
29271,36337,• Кафедральный собор,10857,"Кафедральный Собор, офис",1129636340,"21.229930,54.754365","3.402144,1.063102",0.15,2,3,4,0.666667,1.0,0.0,0,47.049416,25579.327859,1,1,5,540,RU,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
29272,36337,• Кафедральный собор,10857,Кафедральный собор Христа Спасителя,1675621522,"21.229930,54.754365","3.402144,1.063102",0.15,2,4,5,0.500000,0.5,0.0,0,46.953785,25579.327859,1,1,7,720,RU,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
29273,36337,• Кафедральный собор,10857,Кафедральный собор,1129616424,"21.229930,54.754365","3.402144,1.063102",0.35,2,2,4,1.000000,1.0,0.0,0,46.546740,25579.327859,1,1,7,540,RU,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Список добавленных факторов:

In [9]:
features = ['query_len', 'org_name_len', 'org_name_synonyms', 
            'jaccard_index_name', 'jaccard_index_synonyms', 'jaccard_index_address', 
            'geo_match', 'distance_km', 'window_square', 'point_in_window', 
            'rubric_count', 'num_working_days', 'total_working_time'] + \
            [f'region_ohe_{i}' for i in range(11)]
print(len(features))

24


In [10]:
# dataset.to_csv('drive/MyDrive/Colab Notebooks/ML-2/Lab 4/dataset.csv', index=False)
dataset = pd.read_csv('drive/MyDrive/Colab Notebooks/ML-2/Lab 4/dataset.csv')

---

## Ранжирование

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения. Обычно используется три типа:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода, различие между которыми в используемой функции потерь:
  
1. **Pointwise подход**. В этом случае рассматривается *один объект* (в случае поискового ранжирования - конкретный документ) и функция потерь считается только по нему. Любой стандартный классификатор или регрессор может решать pointwise задачу ранжирования, обучившись предсказывать значение таргета. Итоговое ранжирование получается после сортировки документов к одному запросу по предсказанию такой модели.
2. **Pairwise подход**. В рамках данной модели функция потерь вычисляется по *паре объектов*. Другими словами, функция потерь штрафует модель, если отражированная этой моделью пара документов оказалась в неправильном порядке.
3. **Listwise подход**. Этот подход использует все объекты для вычисления функции потерь, стараясь явно оптимизировать правильный порядок.

### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.

*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

В рамках нашей задачи «документом» будет являться организация.

Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание, что разбивать необходимо множество запросов, а не строчки датасета.

In [11]:
def split_dataset(dataset):
    np.random.seed(42)
    queries = dataset['query_id'].unique()
    n_queries = len(queries)
    train_queries = np.random.choice(queries, size=int(0.7 * n_queries), replace=False)
    test_queries = np.array(list(set(queries) - set(train_queries)))

    not_features = ['query', 'region', 'org_name', 'org_id', 'window_center', 'window_size', 'region_code']
    target = ['relevance']
    train = dataset.loc[dataset['query_id'].isin(train_queries)].reset_index(drop=True)
    test = dataset.loc[dataset['query_id'].isin(test_queries)].reset_index(drop=True)

    y_train, y_test = train[target], test[target]
    X_train, X_test = train.drop(not_features + target, axis=1), test.drop(not_features + target, axis=1)
    return X_train, y_train, X_test, y_test

In [12]:
X_train, y_train, X_test, y_test = split_dataset(dataset)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((20097, 25), (20097, 1), (9177, 25), (9177, 1))

Далее рассмотрим несколько подходов предсказания релевантности. Для оценивания качества моделей используйте метрику nDCG на контроле. В случае подбора гиперпараметров используйте кросс-валидацию по 5 блокам, где разбиение должно быть по запросам, а не строчкам датасета.

In [13]:
def dcg_score(y_true, y_score, k=10, gains="exponential"):
    """Discounted cumulative gain (DCG) at rank k"""
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    if gains == "exponential":
        gains = 2 ** y_true - 1
    elif gains == "linear":
        gains = y_true
    else:
        raise ValueError("Invalid gains option.")

    # highest rank is 1 so +2 instead of +1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def ndcg_score(y_true, y_score, k=10, gains="exponential"):
    """Normalized discounted cumulative gain (NDCG) at rank k"""
    best = dcg_score(y_true, y_true, k, gains)
    actual = dcg_score(y_true, y_score, k, gains)
    return actual / best

In [14]:
from sklearn.model_selection import GroupShuffleSplit

def cross_val_indices(X, y):
    query_ids = X['query_id'].values
    gss = GroupShuffleSplit(n_splits=5, train_size=0.7, random_state=42)
    gss.get_n_splits()

    for train_idx, val_idx in gss.split(X, y, query_ids):
        yield train_idx, val_idx

for train_idx, val_idx in cross_val_indices(X_train, y_train):
    print("TRAIN:", train_idx, "VAL:", val_idx)

TRAIN: [    0     1     2 ... 20094 20095 20096] VAL: [  314   315   316 ... 20019 20020 20021]
TRAIN: [    0     1     2 ... 20094 20095 20096] VAL: [   63    64    65 ... 20082 20083 20084]
TRAIN: [    0     1     2 ... 20082 20083 20084] VAL: [   20    21    22 ... 20094 20095 20096]
TRAIN: [    0     1     2 ... 20094 20095 20096] VAL: [   63    64    65 ... 19910 19911 19912]
TRAIN: [    0     1     2 ... 20082 20083 20084] VAL: [   63    64    65 ... 20094 20095 20096]


###  Ранжируем с XGBoost и CatBoost

XGBoost имеет несколько функций потерь для решения задачи ранжирования:
1. **reg:linear** — данную функцию потерь можно использовать для решения задачи ранжирование *pointwise* подходом.
2. **rank:pairwise** — в качестве *pairwise* модели в XGBoost реализован [RankNet](http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf), в котором минимизируется гладкий функционал качества ранжирования: $$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = log(1 + e^{-M}), $$ где $ a(x) $ - функция ранжирования. Суммирование ведется по всем парам объектов, для которых определено отношение порядка, например, для пар документов, показанных по одному запросу. Таким образом функция потерь штрафует за то, что пара объектов неправильно упорядочена.
3. **rank:map, rank:ndcg** — реализация [LambdaRank](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) для двух метрик: [MAP](https://en.wikipedia.org/wiki/Information_retrieval#Mean_average_precision) и **nDCG**. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как **nDCG**,  нужно домножить градиент функционала $ Obj(a) $ на значение $\Delta NDCG_{ij} $ — изменение значения функционала качества при замене $x_i$ на $ x_j$.  Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса *listwise* моделей.

Реализованные в CatBoost ранжирующие функции потерь можной найти [здесь](https://catboost.ai/docs/concepts/loss-functions-ranking.html#groupwise-metrics).

**(3 балла) Задание 2.** Попробуйте различные функции потерь (регрессионные и ранжирующие) для моделей XGBoost и CatBoost. Настройте основные параметры моделей (глубина, кол-во деревьев, скорость обучения, регуляризация).  
Сравните построенные модели с точки зрения метрики nDCG на контроле и проанализируйте полученные результаты:
  - какая модель работает лучше всего для данной задачи? 
  - в чем достоинства/недостатки каждой? 
  - сравните модели между собой: 
   - получается ли сравнимое качество линейного pointwise подхода с остальными моделями? 
   - заметна ли разница в качестве при использовании бустинга с разными функциями потерь?

__XGBoost__

In [15]:
from xgboost import XGBRanker, XGBRegressor
import warnings
warnings.filterwarnings('ignore')

params_grid = {
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.001, 0.01, 0.1],
    'reg_lambda': [0.01, 0.05, 0.1]
}

def fit_evaluate(model, X_train, y_train, X_test, y_test, groups=False):
    if groups:
        groups = X_train.groupby('query_id').size().to_frame('size')['size'].to_numpy()
        model.fit(X_train.drop('query_id', axis=1), y_train, groups)
    else:
        model.fit(X_train.drop('query_id', axis=1), y_train)
    y_pred = model.predict(X_test.drop('query_id', axis=1))
    res = pd.DataFrame({'query_id': X_test['query_id'].values,
                        'y_true': y_test['relevance'].values, 
                        'y_pred': y_pred})
    ndcg = res.groupby('query_id').apply(lambda x: ndcg_score(x['y_true'], x['y_pred'])).mean()
    return ndcg

def cross_validation(model, X_train, y_train, groups=False, n_iter_over_grid=5):
    params_scores = []
    for iter in range(n_iter_over_grid):
        params = {k: np.random.choice(v) for k, v in params_grid.items()}
        model.set_params(**params)
        scores = []
        for train_idx, val_idx in cross_val_indices(X_train, y_train):
            ndcg = fit_evaluate(model, 
                                X_train.iloc[train_idx], y_train.iloc[train_idx], 
                                X_train.iloc[val_idx], y_train.iloc[val_idx], 
                                groups=groups)
            scores.append(ndcg)
        mean_ndcg = np.mean(scores)
        print(f'Iter {iter}, 5-fold cross-val mean NDCG score = {mean_ndcg}')
        params_scores.append((params, mean_ndcg))
    return params_scores

* Pointwise

In [47]:
%%time
model = XGBRegressor(  
    booster='gbtree',
    objective='reg:squarederror',  # reg:linear is now deprecated in favor of reg:squarederror.
    random_state=42, 
)
params_scores = cross_validation(model, X_train, y_train)

Iter 0, 5-fold cross-val mean NDCG score = 0.8093131183818251
Iter 1, 5-fold cross-val mean NDCG score = 0.7898895743282995
Iter 2, 5-fold cross-val mean NDCG score = 0.8068058831699514
Iter 3, 5-fold cross-val mean NDCG score = 0.8008839732788914
Iter 4, 5-fold cross-val mean NDCG score = 0.8205395829231475
CPU times: user 1min 47s, sys: 612 ms, total: 1min 48s
Wall time: 1min 51s


In [48]:
best_params = max(params_scores, key=lambda x: x[1])[0]
model.set_params(**best_params)
ndcg = fit_evaluate(model, X_train, y_train, X_test, y_test)
print(f'NDCG-score with best Pointwise XGBoost: {ndcg}')

NDCG-score with best Pointwise XGBoost: 0.8285247607077414


* Pairwise

In [49]:
%%time
model = XGBRanker(  
    booster='gbtree',
    objective='rank:pairwise',
    random_state=42, 
)
params_scores = cross_validation(model, X_train, y_train, groups=True)

Iter 0, 5-fold cross-val mean NDCG score = 0.844445611062849
Iter 1, 5-fold cross-val mean NDCG score = 0.8236877811982051
Iter 2, 5-fold cross-val mean NDCG score = 0.8233700813822951
Iter 3, 5-fold cross-val mean NDCG score = 0.8180035601498696
Iter 4, 5-fold cross-val mean NDCG score = 0.8182480461878006
CPU times: user 1min 35s, sys: 534 ms, total: 1min 36s
Wall time: 1min 36s


In [50]:
best_params = max(params_scores, key=lambda x: x[1])[0]
model.set_params(**best_params)
ndcg = fit_evaluate(model, X_train, y_train, X_test, y_test, groups=True)
print(f'NDCG-score with best Pairwise XGBoost: {ndcg}')

NDCG-score with best Pairwise XGBoost: 0.8412882703456218


* Listwise

In [51]:
%%time
model = XGBRanker(
    booster='gbtree',
    objective='rank:ndcg',
    random_state=42, 
)
params_scores = cross_validation(model, X_train, y_train, groups=True)

Iter 0, 5-fold cross-val mean NDCG score = 0.9525869131646356
Iter 1, 5-fold cross-val mean NDCG score = 0.9525869131646356
Iter 2, 5-fold cross-val mean NDCG score = 0.9525869131646356
Iter 3, 5-fold cross-val mean NDCG score = 0.9525869131646356
Iter 4, 5-fold cross-val mean NDCG score = 0.9525869131646356
CPU times: user 42.2 s, sys: 422 ms, total: 42.6 s
Wall time: 44.7 s


In [52]:
best_params = max(params_scores, key=lambda x: x[1])[0]
model.set_params(**best_params)
ndcg = fit_evaluate(model, X_train, y_train, X_test, y_test, groups=True)
print(f'NDCG-score with best Listwise XGBoost: {ndcg}')

NDCG-score with best Listwise XGBoost: 0.9594342522908839


__CatBoost__

* Pairwise

In [ ]:
!pip install -q catboost

In [53]:
%%time
from catboost import CatBoostRanker, Pool

model = CatBoostRanker(  
    objective='PairLogit',
    eval_metric='NDCG:hints=skip_train~false',
    random_state=42, 
    n_estimators=400,
)
groups_train, groups_test = X_train['query_id'], X_test['query_id']
model.fit(X_train, y_train, group_id=groups_train, 
          eval_set=(Pool(X_test, y_test, group_id=groups_test)), 
          verbose=50)

0:	learn: 0.7754838	test: 0.7765001	best: 0.7765001 (0)	total: 256ms	remaining: 1m 42s
50:	learn: 0.8654630	test: 0.8602092	best: 0.8622031 (26)	total: 6.52s	remaining: 44.6s
100:	learn: 0.8698709	test: 0.8612319	best: 0.8638932 (94)	total: 14.6s	remaining: 43.2s
150:	learn: 0.8745738	test: 0.8638951	best: 0.8641838 (147)	total: 18.4s	remaining: 30.4s
200:	learn: 0.8787487	test: 0.8661313	best: 0.8667227 (184)	total: 22.2s	remaining: 21.9s
250:	learn: 0.8810588	test: 0.8657683	best: 0.8674094 (218)	total: 25.8s	remaining: 15.3s
300:	learn: 0.8831124	test: 0.8665376	best: 0.8674094 (218)	total: 29.5s	remaining: 9.72s
350:	learn: 0.8884687	test: 0.8651137	best: 0.8674094 (218)	total: 33.2s	remaining: 4.64s
399:	learn: 0.8916430	test: 0.8663683	best: 0.8680889 (379)	total: 36.9s	remaining: 0us

bestTest = 0.8680888682
bestIteration = 379

Shrink model to first 380 iterations.
CPU times: user 37.6 s, sys: 3.11 s, total: 40.7 s
Wall time: 37.1 s


In [54]:
%%time
from catboost import CatBoostRanker, Pool

model = CatBoostRanker(  
    objective='PairLogitPairwise',
    eval_metric='NDCG:hints=skip_train~false',
    random_state=42, 
    n_estimators=400,
)
groups_train, groups_test = X_train['query_id'], X_test['query_id']
model.fit(X_train, y_train, group_id=groups_train, 
          eval_set=(Pool(X_test, y_test, group_id=groups_test)), 
          verbose=50)

0:	learn: 0.7866448	test: 0.7861179	best: 0.7861179 (0)	total: 434ms	remaining: 2m 53s
50:	learn: 0.8679730	test: 0.8595879	best: 0.8602669 (49)	total: 22.9s	remaining: 2m 36s
100:	learn: 0.8790745	test: 0.8668375	best: 0.8681740 (87)	total: 46.7s	remaining: 2m 18s
150:	learn: 0.8842678	test: 0.8657526	best: 0.8681740 (87)	total: 1m 12s	remaining: 2m
200:	learn: 0.8915047	test: 0.8655191	best: 0.8681740 (87)	total: 1m 41s	remaining: 1m 40s
250:	learn: 0.8970256	test: 0.8666456	best: 0.8681740 (87)	total: 2m 3s	remaining: 1m 13s
300:	learn: 0.9003417	test: 0.8706602	best: 0.8706626 (299)	total: 2m 26s	remaining: 48.1s
350:	learn: 0.9024423	test: 0.8691687	best: 0.8707306 (315)	total: 2m 48s	remaining: 23.6s
399:	learn: 0.9070276	test: 0.8680384	best: 0.8707306 (315)	total: 3m 10s	remaining: 0us

bestTest = 0.8707306358
bestIteration = 315

Shrink model to first 316 iterations.
CPU times: user 4min 3s, sys: 7.93 s, total: 4min 11s
Wall time: 3min 10s


* Groupwise

In [55]:
%%time
from catboost import CatBoostRanker, Pool

model = CatBoostRanker(  
    objective='YetiRank',
    eval_metric='NDCG:hints=skip_train~false',
    random_state=42, 
    n_estimators=400,
)
groups_train, groups_test = X_train['query_id'], X_test['query_id']
model.fit(X_train, y_train, group_id=groups_train, 
          eval_set=(Pool(X_test, y_test, group_id=groups_test)), 
          verbose=50)

0:	learn: 0.7968219	test: 0.7828733	best: 0.7828733 (0)	total: 35.2ms	remaining: 14.1s
50:	learn: 0.8758320	test: 0.8642845	best: 0.8647026 (48)	total: 1.84s	remaining: 12.6s
100:	learn: 0.8858550	test: 0.8684131	best: 0.8703074 (78)	total: 3.63s	remaining: 10.8s
150:	learn: 0.8948981	test: 0.8723335	best: 0.8723571 (148)	total: 5.96s	remaining: 9.82s
200:	learn: 0.9012628	test: 0.8715355	best: 0.8730339 (169)	total: 9.37s	remaining: 9.27s
250:	learn: 0.9053203	test: 0.8704287	best: 0.8730339 (169)	total: 12.6s	remaining: 7.5s
300:	learn: 0.9122904	test: 0.8708615	best: 0.8730339 (169)	total: 14.4s	remaining: 4.74s
350:	learn: 0.9163087	test: 0.8711035	best: 0.8730339 (169)	total: 16.2s	remaining: 2.26s
399:	learn: 0.9212188	test: 0.8708277	best: 0.8730339 (169)	total: 19.1s	remaining: 0us

bestTest = 0.8730339211
bestIteration = 169

Shrink model to first 170 iterations.
CPU times: user 24.9 s, sys: 1.06 s, total: 25.9 s
Wall time: 19.2 s


In [56]:
%%time
from catboost import CatBoostRanker, Pool

model = CatBoostRanker(  
    objective='YetiRankPairwise',
    eval_metric='NDCG:hints=skip_train~false',
    random_state=42, 
    n_estimators=400,
)
groups_train, groups_test = X_train['query_id'], X_test['query_id']
model.fit(X_train, y_train, group_id=groups_train, 
          eval_set=(Pool(X_test, y_test, group_id=groups_test)), 
          verbose=50)

0:	learn: 0.8090889	test: 0.7992115	best: 0.7992115 (0)	total: 238ms	remaining: 1m 35s
50:	learn: 0.8508962	test: 0.8481002	best: 0.8489339 (49)	total: 7.74s	remaining: 53s
100:	learn: 0.8642859	test: 0.8568407	best: 0.8572680 (99)	total: 14.6s	remaining: 43.3s
150:	learn: 0.8706662	test: 0.8579602	best: 0.8583196 (138)	total: 21.6s	remaining: 35.6s
200:	learn: 0.8729518	test: 0.8593836	best: 0.8593836 (200)	total: 26.1s	remaining: 25.8s
250:	learn: 0.8749661	test: 0.8601198	best: 0.8601902 (246)	total: 30.3s	remaining: 18s
300:	learn: 0.8778452	test: 0.8629370	best: 0.8629370 (300)	total: 34.4s	remaining: 11.3s
350:	learn: 0.8786160	test: 0.8616796	best: 0.8631229 (301)	total: 38.6s	remaining: 5.38s
399:	learn: 0.8799372	test: 0.8633616	best: 0.8633616 (399)	total: 42.6s	remaining: 0us

bestTest = 0.8633616428
bestIteration = 399

CPU times: user 58.6 s, sys: 1.45 s, total: 1min
Wall time: 42.9 s


__Выводы__

* Наилучшее качество показал XGBoost с Listwise, причем все 5 итераций подбора параметров сошлись в один оптимум, показав одинаковое высокое качество
* Все модели CatBoost показали выше качество, чем Pointwise и Pairwise у XGBoost (PairLogit = 0.868, PairLogitPairwise = 0.870, YetiRank = 0.873, YetiRankPairwise = 0.863)
* CatBoost довольно быстро и качественно обучился (YetiRank = 0.873, 20 секунд), это плюс, но задание параметров модели немного сложнее, чем в XGBoost, аккуратней настраивать надо, это минус
* У XGBoost качество монотонно возрастает по трем методам (Pointwise, Pairwise, Listwise): 0.828 < 0.841 < 0.959; чем лучше мы учитываем порядок в модели, тем качество выше
* Качество линейного подхода (Pointwise) вполне сравнимо с Pairwise подходом, схожие значения
* Меняя функции потерь (objective) мы улучшались в качестве, обращая все больше внимания на общий порядок в выдаче

**(1 балл) Задание 3.** Одним из основных преимуществ CatBoost'a является обработка категориальных факторов «из коробки». Добавьте в датасет различные категориальные факторы из данных и обучите заново CatBoost модели. Улучшилось ли качество?

---

1. Добавим категориальный признак "Язык запроса", проанализировав текст запроса библиотекой __polyglot__
2. Заменим One-Hot региона категориальным признаком

In [ ]:
!pip install -q polyglot
!pip install -q pyicu
!pip install -q pycld2
!pip install -q morfessor

In [20]:
from polyglot.detect import Detector
from polyglot.detect.base import logger
logger.setLevel("ERROR")

def detect_language(text):
    try:
        return Detector(text).language.name
    except Exception:
        return 'UnknownLanguage'

In [21]:
%%time
dataset['query_language'] = dataset['query'].apply(lambda x: detect_language(x))

CPU times: user 1 s, sys: 305 µs, total: 1 s
Wall time: 2.51 s


In [22]:
dataset['query_language'].value_counts().head()

Russian            19116
UnknownLanguage     2284
Turkish             1494
Serbian             1243
Ganda                705
Name: query_language, dtype: int64

In [23]:
dataset_new = dataset.drop([col for col in dataset.columns if col.startswith('region_ohe')], axis=1)
dataset_new['region_code_all'] = dataset_new['org_id'].apply(lambda x: org[str(x)]['address']['region_code'])
print('Unique regions:', dataset_new['region_code_all'].nunique())

Unique regions: 62


In [24]:
dataset_new['query_language'] = dataset_new['query_language'].astype('category')
dataset_new['region_code_all'] = dataset_new['region_code_all'].astype('category')

X_train_cat, y_train, X_test_cat, y_test = split_dataset(dataset_new)

cat_features_indices = np.where(X_train_cat.dtypes == 'category')[0]
print('Cat. features indices:', cat_features_indices)

Cat. features indices: [14 15]


Обучим две CatBoost модели (PairLogit и YetiRank) и сравним качество до/после добавления категориального признака:

In [25]:
%%time
from catboost import CatBoostRanker, Pool

model = CatBoostRanker(  
    objective='PairLogit',
    eval_metric='NDCG:hints=skip_train~false',
    random_state=42, 
    n_estimators=400,
    cat_features=cat_features_indices,
)
groups_train, groups_test = X_train_cat['query_id'], X_test_cat['query_id']
model.fit(X_train_cat, y_train, group_id=groups_train,
          eval_set=(Pool(X_test_cat, y_test, group_id=groups_test, cat_features=cat_features_indices)),
          verbose=50)

Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.7384922	test: 0.7348365	best: 0.7348365 (0)	total: 189ms	remaining: 1m 15s
50:	learn: 0.8629693	test: 0.8551046	best: 0.8559207 (43)	total: 7.01s	remaining: 48s
100:	learn: 0.8713384	test: 0.8627115	best: 0.8627115 (100)	total: 14.8s	remaining: 43.9s
150:	learn: 0.8756434	test: 0.8651426	best: 0.8651856 (148)	total: 21.5s	remaining: 35.5s
200:	learn: 0.8783913	test: 0.8649577	best: 0.8659326 (196)	total: 28.3s	remaining: 28s
250:	learn: 0.8818135	test: 0.8652660	best: 0.8659326 (196)	total: 35s	remaining: 20.8s
300:	learn: 0.8837659	test: 0.8653386	best: 0.8659326 (196)	total: 41.8s	remaining: 13.8s
350:	learn: 0.8873594	test: 0.8700715	best: 0.8701263 (349)	total: 48.6s	remaining: 6.78s
399:	learn: 0.8922005	test: 0.8705299	best: 0.8707669 (394)	total: 55.2s	remaining: 0us

bestTest = 0.8707669268
bestIteration = 394

Shrink model to first 395 iterations.
CPU times: user 1min 20s, sys: 1.72 s, total: 1min 22s
Wall time: 55.4

In [28]:
%%time
from catboost import CatBoostRanker, Pool

model = CatBoostRanker(  
    objective='YetiRank',
    eval_metric='NDCG:hints=skip_train~false',
    random_state=42, 
    n_estimators=400, 
    cat_features=cat_features_indices,
)
groups_train, groups_test = X_train_cat['query_id'], X_test_cat['query_id']
model.fit(X_train_cat, y_train, group_id=groups_train,
          eval_set=(Pool(X_test_cat, y_test, group_id=groups_test, cat_features=cat_features_indices)),
          verbose=50)

Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.7840683	test: 0.7806344	best: 0.7806344 (0)	total: 133ms	remaining: 53.2s
50:	learn: 0.8773506	test: 0.8604620	best: 0.8604620 (50)	total: 6.72s	remaining: 46s
100:	learn: 0.8874058	test: 0.8671830	best: 0.8677091 (93)	total: 9.94s	remaining: 29.4s
150:	learn: 0.8931643	test: 0.8651090	best: 0.8677091 (93)	total: 13.1s	remaining: 21.6s
200:	learn: 0.8996986	test: 0.8678125	best: 0.8686333 (183)	total: 16.2s	remaining: 16.1s
250:	learn: 0.9051852	test: 0.8670769	best: 0.8694377 (223)	total: 19.5s	remaining: 11.5s
300:	learn: 0.9100877	test: 0.8677262	best: 0.8694377 (223)	total: 22.6s	remaining: 7.43s
350:	learn: 0.9139086	test: 0.8693292	best: 0.8695913 (333)	total: 25.7s	remaining: 3.58s
399:	learn: 0.9182471	test: 0.8692524	best: 0.8697781 (381)	total: 28.8s	remaining: 0us

bestTest = 0.8697781343
bestIteration = 381

Shrink model to first 382 iterations.
CPU times: user 46.3 s, sys: 931 ms, total: 47.3 s
Wall time: 29 s


__Вывод:__

* Добавление категориальных признаков улучшило PairLogit, но чуть уменьшило YetiRank
  * Для PairLogit: 0.868 -> 0.871
  * Для YetiRank: 0.873 -> 0.870

#### Пользовательская функция потерь

Библиотека XGBoost позволяет использовать пользовательские функции потерь. Для этого необходимо реализовать функцию, принимающую на вход вектор предсказанных значений и обучающую выборку, и возвращающую градиент и гессиан, посчитанный по входным данным.

Важно отметить, что XGBoost использует диагональную аппроксимацию гессиана, таким образом все недиагональные элементы считаются малозначимыми и приравниваются нулю, поэтому и градиент, и гессиан являются векторами длины размера обучающей выборки.

**(1 балла) Задание 4.** Реализуйте экспоненциальную функцию потерь для XGBoost:
$$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = e^{-M} $$

Обучите модель с помощью данной функции потерь, настройте параметры.

**Комментарии к реализации**

В случае ранжирования XGBoost'у необходимо знать о разбиении всех объектов на группы. В нашем случае в одну группу будут входить документы, соответствующие одному запросу. Функция, считающая градиент и гессиан по данным, должна знать данное разбиение датасета. Однако питоновский интерфейс класса *DMatrix* (в котором хранится датасет) не дает возможности получить это разбиение. В этом случае нужно реализовать функцию потерь в качестве функтора, конструктор которого принимает разбиение на группы в качестве параметра.

Пример реализации своей функции потерь можно найти [в официальной справке](https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html#customized-objective-function) 

In [ ]:
class ExponentialPairwiseLoss(object):
    def __init__(self, groups):
        self.groups = groups
                        
    def __call__(self, preds, dtrain):
        # your code here
        pass

_--->>> Задание отменено, как некорректное_

**(2 балла) Задание 5.** Постройте любую нейросетевую ранжирующую модель
с помощью tensorflow_ranking или используйте DSSM модель на tensorflow/keras/torch по аналогии с моделями на практическом занятии.

In [ ]:
# TODO

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class DSSM(nn.Module):
    
    def __init__(
        self,
        if_dim: int,
        if1_dim: int,
        if2_dim: int,
        final_dim: int,
        dropout: float,
    ):
        super().__init__()
        
        self.item_seq = nn.Sequential(
            nn.Linear(if_dim, if1_dim),
            nn.BatchNorm1d(if1_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(if1_dim, if2_dim),
            nn.BatchNorm1d(if2_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(if2_dim, final_dim),
            nn.BatchNorm1d(final_dim),
            nn.ReLU(),
        )        
        self.cosine = nn.CosineSimilarity()
    
    def forward(
        self,
        item_features,
    ):
        item_vec = self.item_seq(item_features)
        sim = self.cosine(user_vec, item_vec)
        return sim